In [1]:
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.optimization.algorithms import MinimumEigenOptimizer, GroverOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
import numpy as np
from helpers import *
DIR = 'TEST_DATA/'
getPath = lambda fn : DIR+fn

In [2]:
IBMQ.load_account() 
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

/home/tom/anaconda3/envs/Qiskit_Ocean/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [3]:
qps = createModelsFromDir(DIR)
# solve qps with Minimum Eigen Optimizer
quantum_instance = QuantumInstance(backend)
num_iterations=10


In [4]:
qp = qps['test_3']
gas = GroverOptimizer(10, num_iterations=num_iterations, quantum_instance=quantum_instance)
res = gas.solve(qp)

constraint A0_leb0 is infeasible due to substitution
constraint A0_leb1 is infeasible due to substitution


In [5]:
print(res)

optimal function value: -1.0
optimal value: [1. 0. 0.]
status: SUCCESS


# produced 10 Jobs in IBMQ ( 30 minutes )
6043804883ed5a3bac4e7593

In [6]:
cres  = qp.model.solve()
print(cres)

solution for: test_3
objective: -1
x0_2=1
x0_0=1
x0_1=1



In [7]:
qp.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: test_3
// single vars section
dvar bool x0_2;
dvar bool x0_0;
dvar bool x0_1;

minimize
 - x0_2;
 
subject to {
 A0_leb0:
  x0_2 + 2 x0_0 - 10 x0_1 <= 1;
 A0_leb1:
  - x0_2 - 2 x0_0 - 8 x0_1 <= 1;

}


# Test with local simulator

In [9]:
from qiskit import BasicAer
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'))

In [ ]:
for qp_name in sorted(qps.keys(), key = lambda name : int(name[name.rfind('_')+1:])) :
    print(qp_name)
    qp = qps[qp_name]
    gas = GroverOptimizer(8, num_iterations=num_iterations, quantum_instance=quantum_instance)
    res = gas.solve(qp)
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = qp.model.solve()
        if res.fval != cres.get_objective_value():
            print("\n optimal value GAS   "+str(res.fval) + "  ,  cplex:"+ str(cres.get_objective_value()))

test_2
success
* No objective to optimize - searching for a feasible solution
test_3


In [ ]:
# test_3 kills Python Kernel